In [1]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.contrib.slim import fully_connected as fc
import matplotlib.pyplot as plt 
import glob
from collections import OrderedDict
%matplotlib inline
from datetime import datetime
import os
from pathlib import Path
#Ref : https://blog.paperspace.com/implementing-gans-in-tensorflow/

/Users/gongbing/anaconda3/envs/GAN_practice/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/gongbing/anaconda3/envs/GAN_practice/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/gongbing/anaconda3/envs/GAN_practice/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/gongbing/ana

In [1]:
train_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train"
test_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/test"
val_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train"

num_epochs = 50
batch_size=40

def make_dataset(type="train"):
    if type=="train": filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train/*.tfrecords")
    if type=="val":filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/val/*.tfrecords")
    if type=="test":filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/test/*.tfrecords")
    
    def parser(serialized_example):
            seqs = OrderedDict()
            keys_to_features = {
                # 'width': tf.FixedLenFeature([], tf.int64),
                # 'height': tf.FixedLenFeature([], tf.int64),
                'sequence_length': tf.FixedLenFeature([], tf.int64),
                # 'channels': tf.FixedLenFeature([],tf.int64),
                # 'images/encoded':  tf.FixedLenFeature([], tf.string)
                'images/encoded': tf.VarLenFeature(tf.float32)
            }
            parsed_features = tf.parse_single_example(serialized_example, keys_to_features)
            seq = tf.sparse_tensor_to_dense(parsed_features["images/encoded"])
            print("Seq= ",seq.shape)
            images = tf.reshape(seq, [20,64, 64,3], name = "reshape_new")
            seqs["images"] = images
            return seqs
    dataset = tf.data.TFRecordDataset(filenames, buffer_size = 8 * 1024 * 1024)
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.apply(tf.contrib.data.map_and_batch(
            parser, batch_size, drop_remainder = True, num_parallel_calls = None))
    #dataset = dataset.map(parser)
    # num_parallel_calls = None if shuffle else 1  # for reproducibility (e.g. sampled subclips from the test set)
    # dataset = dataset.apply(tf.contrib.data.map_and_batch(
    #    _parser, batch_size, drop_remainder=True, num_parallel_calls=num_parallel_calls)) #  Bing: Parallel data mapping, num_parallel_calls normally depends on the hardware, however, normally should be equal to be the usalbe number of CPUs
    dataset = dataset.prefetch(batch_size)  # Bing: Take the data to buffer inorder to save the waiting time for GPU
    print("dataset",dataset)
    #dataset = dataset.repeat(max_step)
    #dataset = dataset.batch(batch_size)
    #iterator = dataset.make_one_shot_iterator() #One shot iterator will pool all the data once and memery issue
    iterator = dataset.make_initializable_iterator()
    return iterator


X = tf.placeholder(tf.float32,[None,2])
Z = tf.placeholder(tf.float32,[None,2])

NameError: name 'tf' is not defined

In [2]:



class VariantionalAutoencoder(object):

    def __init__(self, learning_rate=1e-4, batch_size=64, n_z=16):
        # Set hyperparameters
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.n_z = n_z
        # Build the graph
        self.build()
        # Initialize paramters
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
        # Summary op
        self.loss_summary = tf.summary.scalar("losses", self.recon_loss)
        self.summary_op = tf.summary.merge_all()
        self.summary_dir = "./"
        self.base_dir = "VAE"
        self.checkpoint_dir =  "VAE" + "/checkpoint"
        Path(self.checkpoint_dir).mkdir(parents=True, exist_ok=True)
        self.train_log_file = self.base_dir + "/train_" + datetime.now().strftime("%Y%m%d-%H%M%S")
        self.val_log_file = self.base_dir + "/val_" + datetime.now().strftime("%Y%m%d-%H%M%S")
        self.train_writer = tf.summary.FileWriter(self.train_log_file, self.sess.graph)
        self.val_writer = tf.summary.FileWriter(self.val_log_file, self.sess.graph)
        self.sess.run(self.train_iterator.initializer)
        self.sess.run(self.val_iterator.initializer)
        
        
        
    def generator(self,Z,hsize=[16, 16],reuse=False):
        with tf.variable_scope("GAN/Generator",reuse=reuse):
            h1 = tf.layers.dense(Z,hsize[0],activation=tf.nn.leaky_relu)
            h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.leaky_relu)
            out = tf.layers.dense(h2,2)
        return out

    def discriminator(self,X,hsize=[16, 16],reuse=False):
        with tf.variable_scope("GAN/Discriminator",reuse=reuse):
            h1 = tf.layers.dense(X,hsize[0],activation=tf.nn.leaky_relu)
            h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.leaky_relu)
            h3 = tf.layers.dense(h2,2)
            out = tf.layers.dense(h3,1)
        return out, h3
       
        
        
    # Build the netowrk and the loss functions
    def build(self):
        
        tf.reset_default_graph()
        self.train_iterator = make_dataset(type="train")
        self.val_iterator = make_dataset(type="val")
        self.test_iterator = make_dataset(type="test")
        self.x = tf.placeholder(tf.float32, [None,20,64,64,3])
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        self.increment_global_step = tf.assign_add(self.global_step,1,name = 'increment_global_step')
        #ARCHITECTURE
        self.mymodel()

        # Loss
        # Reconstruction loss
        # Minimize the cross-entropy loss
        epsilon = 1e-10
        recon_loss = -tf.reduce_sum(
            self.x[:,1:,:,:,:] * tf.log(epsilon+self.x_hat[:,:-1,:,:,:]) + 
            (1-self.x[:,1:,:,:,:]) * tf.log(epsilon+1-self.x_hat[:,:-1,:,:,:]), 
            axis=1
        )
        self.recon_loss = tf.reduce_mean(recon_loss)

        # Latent loss
        # KL divergence: measure the difference between two distributions
        # Here we measure the divergence between 
        # the latent distribution and N(0, 1)
        self.disc_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                                    logits=r_logits,labels=tf.ones_like(r_logits)) + 
                                    tf.nn.sigmoid_cross_entropy_with_logits(
                                        logits=f_logits,labels=tf.zeros_like(f_logits)))
        self.gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,labels=tf.ones_like(f_logits)))

        latent_loss = -0.5 * tf.reduce_sum(
            1 + self.z_log_sigma_sq - tf.square(self.z_mu) - 
            tf.exp(self.z_log_sigma_sq), axis=1)
        self.latent_loss = tf.reduce_mean(latent_loss)

        self.total_loss = self.recon_loss + self.latent_loss
        self.train_op = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate).minimize(self.total_loss,global_step=self.global_step)
        
        # Build a saver
        self.saver = tf.train.Saver(tf.global_variables())
        
        self.losses = {
            'recon_loss': self.recon_loss,
            'latent_loss': self.latent_loss,
            'total_loss': self.total_loss,
        }      # H(x, x_hat) = -\Sigma x*log(x_hat) + (1-x)*log(1-x_hat)

        
        #self.ckpt = tf.train.Checkpoint(model=self.vae_arc2())
        #self.manager = tf.train.CheckpointManager(self.ckpt,self.checkpoint_dir,max_to_keep=3)
        return

    # Execute the forward and the backward pass
    def run_single_step(self):
        global_step = self.sess.run(self.global_step)
        try:
            train_batch = self.sess.run(self.train_iterator.get_next())
            print("Train_batch shape",train_batch["images"].shape)
            #print("global_step",global_step)
            x_hat, train_summary, _, train_losses = self.sess.run([self.x_hat,self.summary_op,self.train_op, self.recon_loss], feed_dict={self.x: train_batch["images"]})
            self.train_writer.add_summary(train_summary, global_step)
            print("x_hat.shape",x_hat.shape)
        except tf.errors.OutOfRangeError:
            print("train out of range error")
        
        try:
            val_batch = self.sess.run(self.val_iterator.get_next())
            val_summary, _, val_losses = self.sess.run([self.summary_op,self.train_op, self.recon_loss], feed_dict={self.x: val_batch["images"]})
            self.val_writer.add_summary(val_summary, global_step)
        except tf.errors.OutOfRangeError:
            print("train out of range error")
        
        return train_losses,val_losses
    
    
    
    
    